<img src="https://media.theresanaiforthat.com/icons/mistral-ai.svg?height=207" width="30" height="30">
<br>
<h1 align='center'>Python Function Call Test</h1>

---

This notebook aims to familiarize myself with a simple example of calling Python functions in the context of a textual conversation with a `Mistral` model.

Function calling is a feature that allows Mistral models to connect to external tools. By integrating these models with external tools such as user-defined functions or APIs, we can easily create applications that respond to specific use cases and solve practical problems. For example, in this guide, we have developed several functions to navigate a database. This allows for more finely-tuned and transparent interactions with the database.

**_This version of the notebook allows you to test function calls with external APIs, including an API requiring authentication._**

There are four steps involved in function calling:

- **User**: specify the tools and the query
- **Model**: generate function arguments if necessary
- **User**: execute the function to obtain the tool results
- **Model**: generate the final response

## Package Loading:

In [104]:
import pandas as pd
import functools
import requests
import json
from mistralai.client import MistralClient
from requests.auth import HTTPBasicAuth
from mistralai.models.chat_completion import ChatMessage

# Création de la base de données :

In [105]:
# Assuming we have the following data
data = {
    'transaction_id': ['T1001', 'T1002', 'T1003', 'T1004', 'T1005'],
    'customer_id': ['C001', 'C002', 'C003', 'C002', 'C001'],
    'payment_amount': [125.50, 89.99, 120.00, 54.30, 210.20],
    'payment_date': ['2021-10-05', '2021-10-06', '2021-10-07', '2021-10-05', '2021-10-08'],
    'payment_status': ['Paid', 'Unpaid', 'Paid', 'Paid', 'Pending']
}

# Create DataFrame
df = pd.DataFrame(data)

# Save DataFrame to CSV file
df.to_csv('items.csv', index=False)

This database will be used by the model to answer questions. To do this, it is necessary to create a series of functions that Mistral will use to query the database.

These functions must be designed in such a way that they can extract a JSON file that can be used by the model.

# Function Declaration:

In [106]:
def retrieve_payment_status(transaction_id: str) -> str:
    data = requests.api.get("http://127.0.0.1:8000/Transaction/" + transaction_id).json()
    if data is not {'detail': 'Item not found'}: 
        return json.dumps({'status': data['Payment_status']}, indent=2)
    return json.dumps({'error': 'transaction id not found.'})

def retrieve_payment_date(transaction_id: str) -> str:
    data = requests.api.get("http://127.0.0.1:8000/Transaction/" + transaction_id).json()
    if data is not {'detail': 'Item not found'}: 
        return json.dumps({'date': data['Payment_status']}, indent=2)
    return json.dumps({'error': 'transaction id not found.'})

def retrieve_payment_max() -> str:
    data = requests.api.get("http://127.0.0.1:8000/Transaction/max").json()
    if data is not {'detail': 'Item not found'}: 
        return json.dumps(data, indent=2)
    return json.dumps({'error': 'transaction id not found.'})

def retrieve_payment_min() -> str:
    data = requests.api.get("http://127.0.0.1:8000/Transaction/min").json()
    if data is not {'detail': 'Item not found'}: 
        return json.dumps(data, indent=2)
    return json.dumps({'error': 'transaction id not found.'})

def retrieve_payment_count(password : str) -> str: #password : str
    data = requests.api.get("http://127.0.0.1:8000/Transaction/count", auth=HTTPBasicAuth("your_username", password)).json()
    if data is not {'detail': 'Item not found'}: 
        return json.dumps({'count' : data}, indent=2)
    return json.dumps({'error': 'transaction id not found.'})

In order for Mistral models to understand and use the functions, it is necessary to describe the specifications of each function using a JSON schema. This involves defining the type, name, description, parameters, and required parameters for each function. In our case, we have two functions to specify, so we need to list two function specifications in a list.

Currently, I do not know of a method to perform this step automatically and quickly.

In [107]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "retrieve_payment_status",
            "description": "Get payment status of a transaction",
            "parameters": {
                "type": "object",
                "properties": {
                    "transaction_id": {
                        "type": "string",
                        "description": "The transaction id.",
                    }
                },
                "required": ["transaction_id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "retrieve_payment_date",
            "description": "Get payment date of a transaction",
            "parameters": {
                "type": "object",
                "properties": {
                    "transaction_id": {
                        "type": "string",
                        "description": "The transaction id.",
                    }
                },
                "required": ["transaction_id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "retrieve_payment_max",
            "description": "Get payment max of all transactions",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "retrieve_payment_min",
            "description": "Get payment min of all transactions",
            "parameters": {
                "type": "object",
                "properties": {},
                "required": [],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "retrieve_payment_count",
            "description": "Get payment count of all transactions",
            "parameters": {
                "type": "object",
                "properties": {
                    "password": {
                        "type": "string",
                        "description": "The password.",
                    }
                },
                "required": ['password'],
            },
        },
    },
]

In [108]:
username = "your_username"
password = "your_password"
data = requests.api.get("http://127.0.0.1:8000/Transaction/count", auth=HTTPBasicAuth(username, password)).json()
data

5

Next, we need to organize the two functions into a dictionary where the keys represent the name of each function and the values correspond to the functions themselves, with the dataframe defined as a parameter. This organization allows us to call each function using its function name.

This step will enable the model to select and call the appropriate function based on the question asked.

In [109]:
names_to_functions = {
    'retrieve_payment_status': functools.partial(retrieve_payment_status),
    'retrieve_payment_date': functools.partial(retrieve_payment_date),
    'retrieve_payment_max': functools.partial(retrieve_payment_max),
    'retrieve_payment_min': functools.partial(retrieve_payment_min),
    'retrieve_payment_count': functools.partial(retrieve_payment_count),
}

We will now initialize the conversation with the Mistral model.

In [110]:
messages = [
    ChatMessage(role="system", content="ta langue de discussion est le français"),
    ChatMessage(role="user", content="combien y'a il de transcation actuellement?")
]

# Function Selection
Initialization of the client and client response call

In [111]:
model = "mistral-large-latest"
api_key="dXhF7z625frqL76yHgWY5PZJ9l0298mg" # ne fonctionne plus après le 31 mars 2024

client = MistralClient(api_key=api_key)
response = client.chat(model=model, messages=messages, tools=tools, tool_choice="auto")
print(response.choices[0].message.content)

To get the count of all transactions, I need to call the `retrieve_payment_count` function. However, it requires a password. Could you please provide the password?


In [112]:
messages.append(response.choices[0].message)
messages.append(ChatMessage(role="user", content="Mon mot de passe est 'your_password'"))
response = client.chat(model=model, messages=messages, tools=tools, tool_choice="auto")
messages.append(response.choices[0].message)

# Execution of Selected Function

In [113]:
tool_call = response.choices[0].message.tool_calls[0]
function_name = tool_call.function.name
function_params = json.loads(tool_call.function.arguments)
print("\nfunction_name: ", function_name, "\nfunction_params: ", function_params)


function_name:  retrieve_payment_count 
function_params:  {'password': 'your_password'}


In [114]:
function_result = names_to_functions[function_name](**function_params)
function_result

'{\n  "count": 5\n}'

In [115]:
messages.append(ChatMessage(role="tool", name=function_name, content=function_result))

response = client.chat(model=model, messages=messages)
messages.append(response.choices[0].message)
print(response.choices[0].message.content)

Il y a actuellement 5 transactions.


**Conclusion**: This notebook demonstrates the feasibility of a data exploration tool based on a textual assistant. Currently, the code execution is manual and local, but it is possible to modify the code to automate this step and provide a seamless experience with the model. Moreover, it is possible to improve the interactions between the database and the model by adding additional functions.